In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

In [ ]:
def create_dataframe(results_file_path, ball_by_ball_file_path):
  results = pd.read_csv(results_file_path)
  ball = pd.read_csv(ball_by_ball_file_path)
  data = ball.merge(results, on='ID', how='left')
  data = data[data['overs'] < 6 ]
  grouped = data.groupby(['ID','innings']).aggregate({'Venue': lambda x: ', '.join(set(x)),
                                                    'total_run': 'sum',
                                                    'batter': lambda x: ', '.join(pd.Series(', '.join(x).split(", ")).drop_duplicates().tolist()),
                                                    'bowler': lambda x: ', '.join(pd.Series(', '.join(x).split(", ")).drop_duplicates().tolist()),
                                                    'BattingTeam': lambda x: ', '.join(set(x)),
                                                    'Team1': lambda x: ', '.join(set(x)),
                                                    'Team2': lambda x: ', '.join(set(x))})
  grouped = grouped.reset_index().drop(['ID'], axis=1)
  final_data = pd.DataFrame()
  final_data['venue'] = grouped['Venue']
  final_data['innings'] = grouped['innings']
  final_data['batting_team'] = grouped['BattingTeam']
  final_data['bowling_team'] = np.where(grouped['BattingTeam']==grouped['Team1'], grouped['Team2'], grouped['Team1'])
  final_data['batsmen'] = grouped['batter']
  final_data['bowlers'] = grouped['bowler']
  mx_batsmen=2
  for i in final_data['batsmen']:
    if len(i.split(",")) > mx_batsmen:
      mx_batsmen=len(i.split(","))
  final_data[["batsman{}".format(i) for i in range(1,mx_batsmen+1)]]=final_data['batsmen'].str.split(', ', mx_batsmen, expand=True)
  mx_bowlers=2
  for i in final_data['bowlers']:
    if len(i.split(",")) > mx_bowlers:
      mx_bowlers=len(i.split(","))
  final_data[["bowler{}".format(i) for i in range(1,mx_bowlers+1)]]=final_data['bowlers'].str.split(', ', mx_bowlers, expand=True)
  final_data = final_data.drop(['batsmen','bowlers','batsman8','batsman7'], axis=1)
  categorical_cols = ['venue', 'batting_team', 'bowling_team']+["bowler{}".format(i) for i in range(1,mx_bowlers+1)]+["batsman{}".format(i) for i in range(1,mx_bowlers+1)]
  return final_data, grouped['total_run'], categorical_cols

In [ ]:
X, y, categorical_cols = create_dataframe("/content/IPL_Matches_Result_2008_2022.csv", '/content/IPL_Ball_by_Ball_2008_2022.csv')

<ipython-input-3-fb794110fc85>:25: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  final_data[["batsman{}".format(i) for i in range(1,mx_batsmen+1)]]=final_data['batsmen'].str.split(', ', mx_batsmen, expand=True)
<ipython-input-3-fb794110fc85>:30: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  final_data[["bowler{}".format(i) for i in range(1,mx_bowlers+1)]]=final_data['bowlers'].str.split(', ', mx_bowlers, expand=True)


In [ ]:
X

,venue,innings,batting_team,bowling_team,batsman1,batsman2,batsman3,batsman4,batsman5,batsman6,bowler1,bowler2,bowler3,bowler4,bowler5,bowler6
0,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,RT Ponting,None,None,None,P Kumar,Z Khan,AA Noffke,None,None,None
1,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,W Jaffer,V Kohli,JH Kallis,CL White,MV Boucher,AB Dinda,I Sharma,AB Agarkar,None,None,None
2,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Kings XI Punjab,PA Patel,ML Hayden,MEK Hussey,None,None,None,B Lee,S Sreesanth,JR Hopes,None,None,None
3,"Punjab Cricket Association Stadium, Mohali",2,Kings XI Punjab,Chennai Super Kings,K Goel,JR Hopes,None,None,None,None,JDP Oram,MS Gony,None,None,None,None
4,Feroz Shah Kotla,1,Rajasthan Royals,Delhi Daredevils,T Kohli,YK Pathan,SR Watson,M Kaif,None,None,GD McGrath,B Geeves,MF Maharoof,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,"Eden Gardens, Kolkata",2,Lucknow Super Giants,Royal Challengers Bangalore,Q de Kock,KL Rahul,M Vohra,DJ Hooda,None,None,Mohammed Siraj,JR Hazlewood,Shahbaz Ahmed,None,None,None
1924,"Narendra Modi Stadium, Ahmedabad",1,Royal Challengers Bangalore,Rajasthan Royals,V Kohli,F du Plessis,RM Patidar,None,None,None,TA Boult,M Prasidh Krishna,None,None,None,None
1925,"Narendra Modi Stadium, Ahmedabad",2,Rajasthan Royals,Royal Challengers Bangalore,YBK Jaiswal,JC Buttler,SV Samson,None,None,None,Mohammed Siraj,JR Hazlewood,GJ Maxwell,Shahbaz Ahmed,None,None
1926,"Narendra Modi Stadium, Ahmedabad",1,Rajasthan Royals,Gujarat Titans,YBK Jaiswal,JC Buttler,SV Samson,None,None,None,Mohammed Shami,Yash Dayal,LH Ferguson,Rashid Khan,None,None


In [ ]:
y

0       61
1       26
2       53
3       63
4       40
        ..
1923    62
1924    46
1925    67
1926    44
1927    31
Name: total_run, Length: 1928, dtype: int64

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

In [ ]:
le = LabelEncoder()
for i in categorical_cols:
    X[i] = le.fit_transform(X[i].astype(str))
print (X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   venue         1928 non-null   int64
 1   innings       1928 non-null   int64
 2   batting_team  1928 non-null   int64
 3   bowling_team  1928 non-null   int64
 4   batsman1      1928 non-null   int64
 5   batsman2      1928 non-null   int64
 6   batsman3      1928 non-null   int64
 7   batsman4      1928 non-null   int64
 8   batsman5      1928 non-null   int64
 9   batsman6      1928 non-null   int64
 10  bowler1       1928 non-null   int64
 11  bowler2       1928 non-null   int64
 12  bowler3       1928 non-null   int64
 13  bowler4       1928 non-null   int64
 14  bowler5       1928 non-null   int64
 15  bowler6       1928 non-null   int64
dtypes: int64(16)
memory usage: 241.1 KB
None


In [ ]:
X

,venue,innings,batting_team,bowling_team,batsman1,batsman2,batsman3,batsman4,batsman5,batsman6,bowler1,bowler2,bowler3,bowler4,bowler5,bowler6
0,19,1,8,16,131,18,148,91,70,33,146,251,10,146,49,5
1,19,2,16,8,108,168,186,49,22,29,11,89,11,146,49,5
2,33,1,0,6,102,94,107,91,70,33,31,206,114,146,49,5
3,33,2,6,0,61,65,128,91,70,33,87,155,178,146,49,5
4,13,1,13,3,146,173,171,68,70,33,66,34,155,146,49,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,12,2,9,16,107,72,102,28,70,33,134,110,249,146,49,5
1924,26,1,16,13,151,42,145,91,70,33,206,138,178,146,49,5
1925,26,2,13,16,158,58,174,91,70,33,134,110,76,209,49,5
1926,26,1,13,5,158,58,174,91,70,33,133,250,142,177,49,5


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

,venue,innings,batting_team,bowling_team,batsman1,batsman2,batsman3,batsman4,batsman5,batsman6,bowler1,bowler2,bowler3,bowler4,bowler5,bowler6
1352,24,1,16,0,102,18,186,91,70,33,53,130,242,146,49,5
141,42,2,3,16,43,163,181,4,70,33,159,181,105,146,49,5
2,33,1,0,6,102,94,107,91,70,33,31,206,114,146,49,5
1190,11,1,8,17,135,44,149,77,70,33,30,5,42,177,49,5
1802,8,1,9,17,65,114,54,77,25,33,30,244,203,146,49,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,19,1,16,4,28,162,7,91,70,33,146,72,223,155,49,5
1294,38,1,13,3,12,35,25,133,70,33,206,204,48,146,49,5
860,13,2,8,3,121,44,128,91,70,33,133,242,144,183,49,5
1459,46,2,13,10,12,58,128,91,70,33,93,24,209,92,49,5


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1542 entries, 1352 to 1126
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   venue         1542 non-null   int64
 1   innings       1542 non-null   int64
 2   batting_team  1542 non-null   int64
 3   bowling_team  1542 non-null   int64
 4   batsman1      1542 non-null   int64
 5   batsman2      1542 non-null   int64
 6   batsman3      1542 non-null   int64
 7   batsman4      1542 non-null   int64
 8   batsman5      1542 non-null   int64
 9   batsman6      1542 non-null   int64
 10  bowler1       1542 non-null   int64
 11  bowler2       1542 non-null   int64
 12  bowler3       1542 non-null   int64
 13  bowler4       1542 non-null   int64
 14  bowler5       1542 non-null   int64
 15  bowler6       1542 non-null   int64
dtypes: int64(16)
memory usage: 204.8 KB


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor


# clf = RandomForestRegressor(random_state=42, n_estimators=135)
clf = RandomForestRegressor(random_state=42, n_estimators=1000, max_depth=12, criterion="friedman_mse", min_samples_split=2, max_features="sqrt", bootstrap=True, ccp_alpha=0.1)

# clf = XGBRegressor(max_depth=1, eta=0.05)
# clf=MLPRegressor(hidden_layer_sizes=())
# clf=LinearRegression()

# r1 = LinearRegression()
# r2 = RandomForestRegressor(n_estimators=10, random_state=1)
# r3 = KNeighborsRegressor()

# clf = VotingRegressor([('clf1', clf1), ('clf2', clf2)])

clf = clf.fit(X_train, y_train)
clf.predict(X_test)

In [ ]:
mean_absolute_error(y_train , clf.predict(X_train))

5.924120419182986

In [ ]:
mean_absolute_error(y_test , clf.predict(X_test))

7.845619216872949